In [11]:
import numpy as np
import pandas as pd

In [81]:
data

,sdid,commentindex,headline,url,guid,commentid,timestamp,thumbs-up,thumbs-down,text,...,y_informative,y_mean,y_controversial,y_disagreement_with_commenter,y_off_topic_with_article,y_sentiment,y_sentiment_neutral,y_sentiment_positive,y_sentiment_negative,y_sentiment_mixed
0,53971,2,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,rjrPtwH5oVVuQnEXX3hf,00003n000000000000000000000000-ed2ae6d0-32ac-4...,1459917444,NaN,NaN,"These things happen , Every job has its dangers.",...,NaN,NaN,NaN,True,True,False,False,False,True,False
1,53971,0,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,VaW6HEsuOFUAIBqjw1k~,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,1459879464,1.0,NaN,Sad to hear such a bad thing. Very dangerous j...,...,NaN,NaN,NaN,NaN,True,False,False,False,False,True
2,53971,1,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,uwQePj970KaMZuW3~9Q9,00002n000000000000000000000000-1c30b878-b717-4...,1459881644,NaN,NaN,Yes..because too many houses in EU look like t...,...,True,False,False,NaN,True,False,True,False,False,False
3,135929,0,This Old Navy Ad Featuring an Interracial Fami...,http://mic.com/articles/142323/this-old-navy-a...,fixyWJivQjEQtPLLVXsu,1462203719963-3eeffb02-faae-4b51-9174-704c57e6...,1462203719,18.0,3.0,"I am frankly quite SICK of the phrase ""shoved ...",...,False,True,False,NaN,True,False,False,False,True,False
4,135929,1,This Old Navy Ad Featuring an Interracial Fami...,http://mic.com/articles/142323/this-old-navy-a...,_TDnK715vO5y0OzZz_n4,00002I000000000000000000000000-7ef2ac58-bd84-4...,1462204643,7.0,2.0,"Ya, I always wonder why the conservatives are ...",...,False,False,False,False,True,False,True,False,False,False
5,135929,2,This Old Navy Ad Featuring an Interracial Fami...,http://mic.com/articles/142323/this-old-navy-a...,JzxshZNmXmdh_2kJN2E9,00003b000000000000000000000000-4a5ccbc3-4a6f-4...,1462239616,1.0,NaN,Great comment!,...,NaN,NaN,NaN,False,True,False,False,True,False,False
6,127970,0,"Patrick Brown, Jailed Because He Could Not Aff...",http://mic.com/articles/141141/patrick-brown-j...,mL_LzSXwMfJ21PfZ~yYv,1461069703997-4fe466e4-91e3-4095-a64b-5d89e8a3...,1461069703,10.0,5.0,The result is a system where being poor become...,...,NaN,NaN,NaN,NaN,NaN,False,True,False,False,False
7,127970,2,"Patrick Brown, Jailed Because He Could Not Aff...",http://mic.com/articles/141141/patrick-brown-j...,lc5tljQ08YooL7Ep3wT3,00003n000000000000000000000000-f1c6653c-a877-4...,1461070200,NaN,NaN,They are also places where you are supposed no...,...,False,False,False,True,NaN,False,True,False,False,False
8,127970,1,"Patrick Brown, Jailed Because He Could Not Aff...",http://mic.com/articles/141141/patrick-brown-j...,fuqaPUBVOAnc0o1gGEIg,00002b000000000000000000000000-498bb84f-8c31-4...,1461069989,1.0,1.0,"Stop trying to make sense, it only confuses pe...",...,False,True,False,True,NaN,False,False,False,True,False
9,127970,3,"Patrick Brown, Jailed Because He Could Not Aff...",http://mic.com/articles/141141/patrick-brown-j...,mL_LzSXwMfJ21PfZ~yYv,00004g000000000000000000000000-1524cf7b-652d-4...,1461079572,NaN,1.0,Madd...sorry - I can't help it. Grim....are yo...,...,False,False,False,True,NaN,False,True,False,False,False


In [3]:
data = pd.read_csv('../data/raw/ydata-ynacc-v1_0_expert_annotations.tsv', sep='\t')

In [36]:
data['y_persuasive'] = data['persuasiveness'].str.contains('Persuasive')
data['y_audience'] = data['intendedaudience'].str.contains('Reply to a specific commenter')
data['y_agreement_with_commenter'] = data['commentagreement'].str.contains('Agreement with commenter')
data['y_informative'] = data['tone'].str.contains('Informative')
data['y_mean'] = data['tone'].str.contains('Mean')
data['y_controversial'] = data['tone'].str.contains('Controversial')
data['y_disagreement_with_commenter'] = data['commentagreement'].str.contains('Disagreement with commenter')
data['y_off_topic_with_article'] = data['topic'].str.contains('Off-topic with article')
data['y_sentiment_neutral'] = data['sentiment'].str.contains('neutral')
data['y_sentiment_positive'] = data['sentiment'].str.contains('positive')
data['y_sentiment_negative'] = data['sentiment'].str.contains('negative')
data['y_sentiment_mixed'] = data['sentiment'].str.contains('mixed')

In [39]:
tiny = data.drop_duplicates(['y_persuasive', 'y_audience', 'y_agreement_with_commenter', 'y_informative', 'y_mean', 'y_controversial', 'y_disagreement_with_commenter', 'y_off_topic_with_article', 'y_sentiment_neutral', 'y_sentiment_positive', 'y_sentiment_negative', 'y_sentiment_mixed'])

In [72]:
def valid_dataset(df):
    return np.sum(df['y_persuasive']) >= 1 and np.sum(df['y_audience']) >= 1 and np.sum(df['y_agreement_with_commenter']) >= 1 and np.sum(df['y_informative']) >= 1 and np.sum(df['y_mean']) >= 1 and np.sum(df['y_controversial']) >= 1 and np.sum(df['y_disagreement_with_commenter']) >= 1 and np.sum(df['y_off_topic_with_article']) >= 1 and np.sum(df['y_sentiment_neutral']) >= 1 and np.sum(df['y_sentiment_positive']) >= 1 and np.sum(df['y_sentiment_negative']) >= 1 and np.sum(df['y_sentiment_mixed']) >= 1

In [73]:
tmp = tiny.sample(10)
while not valid_dataset(tmp):
    tmp = tiny.sample(10)

In [77]:
tmp = tmp.fillna(False)
tmp.to_csv('../data/datasets/Tiny/train/comments.csv', index=False)
tmp.to_csv('../data/datasets/Tiny/test/comments.csv', index=False)

In [78]:
tmp['y_persuasive']

13715     True
15090    False
2027     False
20949     True
11389    False
86       False
1241      True
254      False
1242     False
475      False
Name: y_persuasive, dtype: bool

In [80]:
save_data = data.fillna(False)
save_data.to_csv('../data/datasets/YNACC/train/comments.csv', index=False)
save_data.to_csv('../data/datasets/YNACC/test/comments.csv', index=False)